In [1]:
!pip -q install pyomo
!apt-get -q install -y -qq coinor-cbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3

**Question 1: Course Registration**

In [2]:
from pyomo.environ import *
import numpy as np

In [3]:
# N: no. of variables
# M: no. of constraints

N, M = 11, 11

# vector of cost cofficient
c = np.array([3, 2, 6, 5, 5, 4, 6, 8, 8, 5, 4])    # credits of all course

# matrix of matrix for coefficient of constraints and vector of resource constraints
A = np.array([
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],   # sum(x_i) <= 9
              [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],   # x_2 + x_7 <= 1
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],   # x_4 + x_9 <= 1
              [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],   # x_2 + x_8 <= 1
              [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],  # x_10 - x_0 <= 0
              [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],   # x_0 + x_1 + x_2 <= 2
              [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],   # x_5 + x_6 <= 1
              [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],   # x_3 + x_6 + x_9 <= 2

              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],   # sum(x_i) >= 5
              [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],   # x_0 + x_1 + x_2 >= 1
              c,                                   # sum(c_i*x_i) >= 30
              ])

b = np.array([9, 1, 1, 1, 0, 2, 1, 2, 5, 1, 30])

In [4]:
# defining empty pyomo model as ILP_Model
MILP_Model = ConcreteModel()

# variable are defined as NonNegativeIntegers
MILP_Model.x = Var(np.arange(N), domain = Binary, bounds= (0,1))

In [5]:
MILP_Model.constraints = ConstraintList()

[MILP_Model.constraints.add(sum(A[i][j]*MILP_Model.x[j] for j in range(N)) <= b[i]) for i in range(8)]

[MILP_Model.constraints.add(sum(A[i][j]*MILP_Model.x[j] for j in range(N)) >= b[i]) for i in range(8, 11)]

#[MILP_Model.constraints.add(sum(A[-1][j]*MILP_Model.x[j]*c[j] for j in range(N)) >= b[-1])]

In [6]:
# objective for above formulation defined
MILP_Model.Objective = Objective(expr = sum(c[j]*MILP_Model.x[j] for j in range(N)), sense = maximize)

# printing your formulation
MILP_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None 

In [7]:
optCBC = SolverFactory('cbc')

resultCBC = optCBC.solve(MILP_Model)
print(resultCBC)

def OptResults(Model):

  print('\nObjective = ', Model.Objective())

  print('\nDecision Variables')
  for i in np.arange(N):
    print('x[',i,'] = ', Model.x[i].value)
  print('\nConstraints')
  Model.constraints.display()

MILP_ModelSolution = OptResults(MILP_Model)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.06491231918334961
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  41.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0

Constraints
constraints : Size=11
    Ke

In [8]:
# Model
Linear_Model = ConcreteModel()

# Variables
Linear_Model.x = Var(np.arange(N), domain = Reals, bounds= (0,1))

# Constraints
Linear_Model.constraints = ConstraintList()

[Linear_Model.constraints.add(sum(A[i][j]*Linear_Model.x[j] for j in range(N)) <= b[i]) for i in range(8)]

[Linear_Model.constraints.add(sum(A[i][j]*Linear_Model.x[j] for j in range(N)) >= b[i]) for i in range(8, 11)]

# Objective
Linear_Model.Objective = Objective(expr = sum(c[j]*Linear_Model.x[j] for j in range(N)), sense = maximize)

# Print the built Model
Linear_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

1 Var Declarations
    x : Size=11, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None 

In [9]:
# Solve
optCBC = SolverFactory('cbc')

# Results
resultCBC = optCBC.solve(Linear_Model)
print(resultCBC)

# Value obtained for objective and constraints
Linear_Model_Solution = OptResults(Linear_Model)


Problem: 
- Name: unknown
  Lower bound: 41.0
  Upper bound: 41.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.02521228790283203
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  41.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0

Constraints
constraints : Size=11
    Key : Lower : Body : Upper
      1 :  None :  8.0 :   9.0
      

In [10]:
# N: no. of variables
# M: no. of constraints

N, M = 12, 12

# vector of cost cofficient
c_new = np.append(c, 9)    # credits of all course

# Add new coulm for C11 coefficients
C11_col = np.zeros(A.shape[1])
C11_col = np.expand_dims(C11_col, 1)
A_new = np.hstack((A, C11_col))

# Add credits of C11 for sum(c_i*x_i) >= 30 constraint
A_new[-1, -1] = 9

# Add coeeficients of the new constraint
A_new = np.vstack((A_new, [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1]))

# Update b
b_new = np.append(b, 1)

In [11]:
# Define model
MILP_Model = ConcreteModel()

# variables
MILP_Model.x = Var(np.arange(N), domain = Binary, bounds= (0,1))

# Constraints
MILP_Model.constraints = ConstraintList()

[MILP_Model.constraints.add(sum(A_new[i][j]*MILP_Model.x[j] for j in range(N)) <= b_new[i]) for i in range(8)]

[MILP_Model.constraints.add(sum(A_new[i][j]*MILP_Model.x[j] for j in range(N)) >= b_new[i]) for i in range(8, 11)]

# new constraint for C11
[MILP_Model.constraints.add(sum(A_new[-1][j]*MILP_Model.x[j] for j in range(N)) <= b_new[-1])]

# Objective
MILP_Model.Objective = Objective(expr = sum(c_new[j]*MILP_Model.x[j] for j in range(N)), sense = maximize)

# Print the built Model
MILP_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 

In [12]:
# Solve
optCBC = SolverFactory('cbc')

# Results
resultCBC = optCBC.solve(MILP_Model)
print(resultCBC)

# Value obtained for objective and constraints
MILP_Model_Solution = OptResults(MILP_Model)


Problem: 
- Name: unknown
  Lower bound: 38.0
  Upper bound: 38.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of binary variables: 12
  Number of integer variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03386068344116211
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  38.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  1.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0

Constraints
constraints 

In [13]:
# Define model
Linear_Model = ConcreteModel()

# variables
Linear_Model.x = Var(np.arange(N), domain = Reals, bounds= (0,1))

# Constraints
Linear_Model.constraints = ConstraintList()

[Linear_Model.constraints.add(sum(A_new[i][j]*Linear_Model.x[j] for j in range(N)) <= b_new[i]) for i in range(8)]

[Linear_Model.constraints.add(sum(A_new[i][j]*Linear_Model.x[j] for j in range(N)) >= b_new[i]) for i in range(8, 11)]

# new constraint for C11
[Linear_Model.constraints.add(sum(A_new[-1][j]*Linear_Model.x[j] for j in range(N)) <= b_new[-1])]

# Objective
Linear_Model.Objective = Objective(expr = sum(c_new[j]*Linear_Model.x[j] for j in range(N)), sense = maximize)

# Print the built Model
Linear_Model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    x : Size=12, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 

In [14]:
# Solve
optCBC = SolverFactory('cbc')

# Results
resultCBC = optCBC.solve(Linear_Model)
print(resultCBC)

# Value obtained for objective and constraints
Linear_Model_Solution = OptResults(Linear_Model)


Problem: 
- Name: unknown
  Lower bound: 38.0
  Upper bound: 38.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 12
  Number of nonzeros: 12
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.026879549026489258
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


Objective =  38.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  0.0
x[ 2 ] =  1.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  0.0
x[ 6 ] =  1.0
x[ 7 ] =  0.0
x[ 8 ] =  0.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0

Constraints
constraints : Size=12
    Key : Lower : Body : Upper
      1 :  None :  6.